In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

In [ ]:
x = np.linspace(-5,5,200)

y_relu = tf.nn.relu(x)
y_sigmoid = tf.nn.sigmoid(x)
y_tanh = tf.nn.tanh(x)
y_softplus = tf.nn.softplus(x)

plt.figure(1, figsize=(8, 6))
plt.subplot(221)
plt.plot(x, y_relu, c='red', label='relu')
plt.ylim((-1, 5))
plt.legend(loc='best')

plt.subplot(222)
plt.plot(x, y_sigmoid, c='red', label='sigmoid')
plt.ylim((-0.2, 1.2))
plt.legend(loc='best')

plt.subplot(223)
plt.plot(x, y_tanh, c='red', label='tanh')
plt.ylim((-1.2, 1.2))
plt.legend(loc='best')

plt.subplot(224)
plt.plot(x, y_softplus, c='red', label='softplus')
plt.ylim((-0.2, 6))
plt.legend(loc='best')

plt.show()

In [ ]:
#tf.random.set_seed(1234)
np.random.seed(1234)

# data initialization
# linspace creates a list with continous intervals of range -1 to 1
# newaxis increases the dimension i.e. changes (1,100) to (100,1)
x = np.linspace(-1,1,100)[:, np.newaxis]
noise = np.random.normal(0,0.1,size=x.shape)
y = np.power(x,2) + noise

# plot the data
plt.scatter(x,y)
plt.show()

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

tf_x = tf.placeholder(tf.float32, x.shape)
tf_y = tf.placeholder(tf.float32, y.shape)

# hidden layer
l1 = tf.layers.dense(tf_x,1000,tf.nn.relu)
# output layer
#l2 = tf.layers.dense(l1,100,tf.nn.relu)
output = tf.layers.dense(l1, 1)

# compute cost
loss = tf.losses.mean_squared_error(tf_y, output)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5)
train_op = optimizer.minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

plt.ion()

for step in range(100):
    _, l, pred = sess.run([train_op, loss, output], {tf_x: x, tf_y: y})
    if step%5 == 0:
        plt.cla()
        plt.scatter(x,y)
        plt.plot(x, pred, 'r-', lw=5)
        plt.text(0.5, 0, 'loss=%.4f' % l, fontdict = {'size': 20, 'color':'red'})
        plt.pause(0.1)
        
plt.ioff()
plt.show()

In [ ]:
import pandas as pd
ML_input_data = pd.read_csv('./ML_input_data.csv')

In [ ]:
retail_950s = ML_input_data[(ML_input_data['Forecast Sales Model Number'] == "950") &
                            (ML_input_data['Sales Channel'] == "Retail") &
                            (ML_input_data['Global Market'] == "AMN")] \
                            .reset_index() \
                            [['Final Usage Hours', 'Relative Price Change']]

#retail_950s = retail_950s.to_numpy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler



def scale_variable(feature):
    product = feature.values
    product = product.reshape(-1,1)
    product_scaler = MinMaxScaler(feature_range=(0,1))
    product = product_scaler.fit_transform(product)
    #product = product.reshape(-1,) # reshape it back
    
    return product, product_scaler

x, x_scaler = scale_variable(retail_950s['Final Usage Hours'].head(1000))

y, y_scaler = scale_variable(retail_950s['Relative Price Change'].head(1000))


In [ ]:
x.shape

# Attempted TensorFlow 2 Version

In [ ]:
model = keras.Sequential([
    keras.layers.Reshape(target_shape=(100,), input_shape=(100, 1)),
    keras.layers.Dense(units=1, activation='relu'),
    keras.layers.Dense(units=12, activation='relu'),
    keras.layers.Dense(units=1, activation='softmax')
])

 

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
train_dataset = (x, y)
val_dataset = (x,y)


history = model.fit(
    x = x,
    y = y,
    epochs=10, 
    steps_per_epoch=10,
    validation_data=val_dataset, 
    validation_steps=2
)

In [ ]:
x